In [ ]:
import os
import pandas as pd
from neu_perm.config import RESULTS_DIR

# df = pd.read_csv('llama-3.2-1b_squad.csv')

def ret_dfs(paths, base_path=None):
    if base_path is None:
        base_path = RESULTS_DIR

    dfs = []
    for path in paths:
        df = pd.read_csv(os.path.join(base_path, path))
        dfs.append(df)

    df = pd.concat(dfs, ignore_index=True)
    return df

paths = [
    # 'llama-3.2-1b_squad.csv',

    'densenet121_imagenet12.csv',
    'resnet50_imagenet12.csv',
    'resnet101_imagenet12.csv',
]
df = ret_dfs(paths)


df.drop(columns=['dataset',], inplace=True)
# df.drop(columns=['model_name', 'dataset',], inplace=True)

df_original = df[df['method'] == 'original'].copy()
df_noise = df[df['method'] == 'noise'].copy()
df_prune = df[df['method'] == 'prune'].copy()
df_neuperm = df[df['method'] == 'neuperm'].copy()

df_prune['amount'] = df_prune['method_kwargs'].apply(lambda x: eval(x)['amount'])
df_prune.drop(columns=['method_kwargs'], inplace=True)

df_noise['eps'] = df_noise['method_kwargs'].apply(lambda x: eval(x)['eps'])
df_noise.drop(columns=['method_kwargs'], inplace=True)
# df_noise

In [ ]:
df_noise.groupby(['model_name', 'eps', ])['accuracy'].agg(['mean', 'std']).reset_index()

,model_name,eps,mean,std
0,densenet121,0.0001,0.744262,0.000296
1,densenet121,0.0010,0.741466,0.001425
2,densenet121,0.0100,0.134574,0.062366
3,densenet121,0.1000,0.001000,0.000000
4,resnet101,0.0001,0.818930,0.000125
5,resnet101,0.0010,0.795310,0.038281
6,resnet101,0.0100,0.002770,0.003277
7,resnet101,0.1000,0.001000,0.000000
8,resnet50,0.0001,0.808308,0.000105
9,resnet50,0.0010,0.479670,0.260018


In [8]:
df_prune.groupby(['model_name', 'amount', ])['accuracy'].agg(['mean', 'std']).reset_index()

,model_name,amount,mean,std
0,densenet121,0.01,0.695960,0.014802
1,densenet121,0.05,0.361592,0.057529
2,resnet101,0.01,0.725004,0.034807
3,resnet101,0.05,0.189702,0.086773
4,resnet50,0.01,0.626532,0.085746
5,resnet50,0.05,0.028992,0.024350


In [9]:
df_original

,model_name,accuracy,time,method,method_kwargs


In [15]:
df_neuperm.groupby(['model_name',])['accuracy'].agg(['mean', 'std'])

,mean,std
model_name,,
densenet121,0.744374,0.000016
resnet101,0.819034,0.000037
resnet50,0.808426,0.000035
